### 練習問題2.52
上で述べた各レベルに⼿を加え、  
図2.9に⽰したwaveによるsquare-limitを変更せよ。  
具体的には、次のようなものである。

- a.練習問題2.49のwave基本ペインタに線分をいくつか加えよ(例えば、笑顔を追加するなど)。
- b.corner-splitによって構築されるパターンを変更せよ(例えば、up-splitやright-splitの画像を⼆つではなく⼀つ だけ使うようにする)。
- c.square-of-fourを使うほうのsquare-limitを修正し、四隅を違うパターンで組み⽴てるようにせよ(例えば、正⽅形の四隅で⼤きなロジャーズ⽒が外を向くようにするなど)。 


回答は割愛する。